# Question 1

Importing the libraries

In [74]:
import pandas as pd
import numpy as np
import requests 
from bs4 import BeautifulSoup as bs
import geocoder
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors

Scrapping and viewing the raw data

In [2]:
# Getting the URL to the page
url = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
wiki_url = requests.get(url).text
# Read the url with pandas
toronto_data = bs(wiki_url, 'html5')
# The original data scrapped 
toronto_data

<!DOCTYPE html>
<html class="client-nojs" dir="ltr" lang="en"><head>
<meta charset="utf-8"/>
<title>List of postal codes of Canada: M - Wikipedia</title>
<script>document.documentElement.className="client-js";RLCONF={"wgBreakFrames":!1,"wgSeparatorTransformTable":["",""],"wgDigitTransformTable":["",""],"wgDefaultDateFormat":"dmy","wgMonthNames":["","January","February","March","April","May","June","July","August","September","October","November","December"],"wgRequestId":"907f6da5-78eb-4d91-b75d-c4ccf99c058f","wgCSPNonce":!1,"wgCanonicalNamespace":"","wgCanonicalSpecialPageName":!1,"wgNamespaceNumber":0,"wgPageName":"List_of_postal_codes_of_Canada:_M","wgTitle":"List of postal codes of Canada: M","wgCurRevisionId":1032600019,"wgRevisionId":1032600019,"wgArticleId":539066,"wgIsArticle":!0,"wgIsRedirect":!1,"wgAction":"view","wgUserName":null,"wgUserGroups":["*"],"wgCategories":["Articles with short description","Short description is different from Wikidata","Communications in Ontario","

Convert our data into a dataframe

In [3]:
# Scrapping our data and Ignoring the 
contents=[]
table = toronto_data.find('table')
for row in table.find_all('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighbourhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        contents.append(cell)

For the code above we already did not Scapped the one where Borough is 'Not assigned'

In [4]:
# Checking our new dataframe
toronto_df = pd.DataFrame(contents)
toronto_df.head(5)

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


**Grouping the Postal codes which have two neighbours together**

In [5]:
toronto_data = toronto_df.groupby(['PostalCode'])
toronto_df.head(5)

,PostalCode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


**Checking for number of records where Neighbourhood is "Not assigned"**

In [6]:
toronto_df.Neighbourhood.str.count("Not assigned").sum()

0

**Printing the SHAPE of the dataset**

In [7]:
toronto_df.shape

(103, 3)

# Question 2

In [76]:
pip install geocoder

Note: you may need to restart the kernel to use updated packages.


In [9]:
# Getting the Lat and Long 
def latlong(postalcode):
    lat_long_coord = None
    while(lat_long_coord is None):
        g = geocoder.arcgis('{}, Toronto, Ontario'.format(postalcode))
        lat_long_coord = g.latlng
    return lat_long_coord

In [10]:
postalcode = toronto_df['PostalCode']    
coordinates = [latlong(postalcode) for postalcode in postalcode.tolist()]

In [11]:
# Creating a new dataframe using Geocoder
df_geo = pd.DataFrame(coordinates, columns=['Latitude', 'Longitude'])
toronto_df['Latitude'] = df_geo['Latitude']
toronto_df['Longitude'] = df_geo['Longitude']

In [12]:
# Viewing our newly created table
toronto_df

,PostalCode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.75245,-79.32991
1,M4A,North York,Victoria Village,43.73057,-79.31306
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.72327,-79.45042
4,M7A,Queen's Park,Ontario Provincial Government,43.66253,-79.39188
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.65319,-79.51113
99,M4Y,Downtown Toronto,Church and Wellesley,43.66659,-79.38133
100,M7Y,East TorontoBusiness reply mail Processing Cen...,Enclave of M4L,43.64869,-79.38544
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.63278,-79.48945


# Question 3

In [61]:
from geopy.geocoders import Nominatim
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The coordinates of Toronto are {}, {}.'.format(latitude, longitude))
df = toronto_df

The coordinates of Toronto are 43.6534817, -79.3839347.


In [62]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)
for lat, lng, borough, neighborhood in zip(df['Latitude'], df['Longitude'], df['Borough'], df['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=4,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#87cefa',
        fill_opacity=0.5,
        parse_html=False).add_to(map_toronto)
map_toronto

In [63]:
# Four Square Credentials
CLIENT_ID = '3HA5I1NALIQBX0X2QJAWUTAIJNG4N2GOWJEXWVWFP4FYAO3V'    
CLIENT_SECRET = 'HSMUV2PIJJG3RESNP4U53NKWE2XDP0DM3M5FEMPAZFYJAJ5X'
VERSION = '20180604'

In [64]:
# Function that gets neighbouring venues
limit = 20
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, limit)
        
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        venues_list.append([( name, lat, lng, v['venue']['name'], v['venue']['location']['lat'], v['venue']['location']['lng'], v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 'Neighbourhood Latitude', 'Neighbourhood Longitude', 'Venue', 'Venue Latitude', 'Venue Longitude', 'Venue Category']
    
    return(nearby_venues)

In [65]:
toronto_venues = getNearbyVenues(names=df['Neighbourhood'], latitudes=df['Latitude'],longitudes=df['Longitude'])

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Ontario Provincial Government
Islington Avenue
Malvern, Rouge
Don Mills North
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills South
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
The Danforth  East
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmount Park
Bayview Village
Downsview East
The Danforth

In [33]:
toronto_venues.shape

(1099, 7)

We have 1099 records and 7 columns


In [34]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.75245,-79.32991,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.75245,-79.32991,KFC,43.754387,-79.333021,Fast Food Restaurant
2,Parkwoods,43.75245,-79.32991,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,Victoria Village,43.73057,-79.31306,Wigmore Park,43.731023,-79.310771,Park
4,Victoria Village,43.73057,-79.31306,Memories of Africa,43.726602,-79.312427,Grocery Store


In [66]:
toronto_venues.groupby('Neighbourhood').head()

,Neighbourhood,Neighbourhood Latitude,Neighbourhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.75245,-79.32991,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.75245,-79.32991,KFC,43.754387,-79.333021,Fast Food Restaurant
2,Parkwoods,43.75245,-79.32991,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,Victoria Village,43.73057,-79.31306,Wigmore Park,43.731023,-79.310771,Park
4,Victoria Village,43.73057,-79.31306,Memories of Africa,43.726602,-79.312427,Grocery Store
...,...,...,...,...,...,...,...
1086,"Mimico NW, The Queensway West, South of Bloor,...",43.62513,-79.52681,Royal Meats BBQ,43.625353,-79.529073,Burger Joint
1087,"Mimico NW, The Queensway West, South of Bloor,...",43.62513,-79.52681,K & B Sushi,43.622269,-79.522485,Sushi Restaurant
1088,"Mimico NW, The Queensway West, South of Bloor,...",43.62513,-79.52681,Xtreme Couture,43.624799,-79.528626,Gym / Fitness Center
1089,"Mimico NW, The Queensway West, South of Bloor,...",43.62513,-79.52681,Mangosteens Thai Cuisine,43.621419,-79.524771,Thai Restaurant


Check max venue categories

In [37]:
toronto_venues.groupby('Venue Category').max()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude
Venue Category,,,,,,
American Restaurant,"Toronto Dominion Centre, Design Exchange",43.66213,-79.33497,The Ace,43.661937,-79.335938
Antique Shop,Studio District,43.66213,-79.33497,Guff,43.662423,-79.334265
Art Gallery,"Little Portugal, Trinity",43.66505,-79.37804,Ryerson Image Centre,43.668865,-79.379460
Art Museum,Central Bay Street,43.65609,-79.38493,Textile Museum of Canada,43.654396,-79.386500
Arts & Crafts Store,Woodbine Heights,43.76774,-79.30682,The Workroom,43.767245,-79.310253
...,...,...,...,...,...,...
Vegetarian / Vegan Restaurant,"University of Toronto, Harbord",43.66311,-79.37306,Rosalinda,43.666755,-79.374453
Video Game Store,"Clarks Corners, Tam O'Shanter, Sullivan",43.78491,-79.29722,Just Games,43.784049,-79.292819
Vietnamese Restaurant,Studio District,43.79452,-79.26708,Saigon Lotus Restaurant,43.797819,-79.270737


There are 207 different venues

**ONE HOT ENCODING**

In [67]:
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]
toronto_onehot.head()

,Neighbourhood,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [68]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped.head(5)

,Neighbourhood,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Dealership,Auto Garage,...,Theme Restaurant,Toy / Game Store,Trail,Train Station,Turkish Restaurant,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Yoga Studio
0,Agincourt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.083333,0.0,0.0
1,"Alderwood, Long Branch",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
2,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
3,Bayview Village,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.4,0.0,0.0,0.0,0.0,0.000000,0.0,0.0
4,"Bedford Park, Lawrence Manor East",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0


In [44]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

In [70]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighbourhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Agincourt,Chinese Restaurant,Hong Kong Restaurant,Supermarket,Sushi Restaurant,Bakery,Badminton Court,Department Store,Shopping Mall,Grocery Store,Vietnamese Restaurant
1,"Alderwood, Long Branch",Gym,Performing Arts Venue,Convenience Store,Pub,Dessert Shop,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Dog Run
2,"Bathurst Manor, Wilson Heights, Downsview North",Business Service,Yoga Studio,Comfort Food Restaurant,Falafel Restaurant,Event Space,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Dog Run
3,Bayview Village,Trail,Park,Construction & Landscaping,Dog Run,Dessert Shop,Event Space,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
4,"Bedford Park, Lawrence Manor East",Sandwich Place,Italian Restaurant,Coffee Shop,Pharmacy,Liquor Store,Restaurant,Juice Bar,Fast Food Restaurant,Indian Restaurant,Pub


**CLUSTERING OUR DATA**

In [71]:
from sklearn.cluster import KMeans
# Setting cluster number
k_num_clusters = 7
toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)
kmeans = KMeans(n_clusters=k_num_clusters, random_state=0).fit(toronto_grouped_clustering)
kmeans

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=7, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=0, tol=0.0001, verbose=0)

In [77]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
toronto_merged = df
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood')
toronto_merged.head() 

,PostalCode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,Cluster Label,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.75245,-79.32991,1.0,1.0,Park,Fast Food Restaurant,Food & Drink Shop,Diner,Event Space,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Dog Run
1,M4A,North York,Victoria Village,43.73057,-79.31306,1.0,1.0,Park,Nail Salon,German Restaurant,Grocery Store,Art Museum,Discount Store,Farm,Falafel Restaurant,Event Space,Electronics Store
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.65512,-79.36264,0.0,0.0,Coffee Shop,Breakfast Spot,Bakery,Gym / Fitness Center,Food Truck,Italian Restaurant,Event Space,Electronics Store,Distribution Center,Pub
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.72327,-79.45042,0.0,0.0,Clothing Store,Cosmetics Shop,Electronics Store,Men's Store,Food Court,Jewelry Store,Kitchen Supply Store,Chocolate Shop,Furniture / Home Store,Toy / Game Store
4,M7A,Queen's Park,Ontario Provincial Government,43.66253,-79.39188,0.0,0.0,Coffee Shop,Café,Mediterranean Restaurant,Falafel Restaurant,Sandwich Place,Park,Burrito Place,Gastropub,Bank,Italian Restaurant


**VISUALIZING THE DATA VALUES**

In [79]:
toronto_merged = toronto_merged.dropna(subset=['Cluster Labels'])

In [80]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(k_num_clusters)
ys = [i + x + (i*x)**2 for i in range(k_num_clusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup('Cluster ' + str(int(cluster) +1) + '\n' + str(poi) , parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[int(cluster-1)],
        fill=True,
        fill_color=rainbow[int(cluster-1)]
        ).add_to(map_clusters)
        
map_clusters